# Deploying on Flask with Colab

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import socket
print(socket.gethostbyname(socket.gethostname()))

172.28.0.2


# Running Flask as a Background Process

In [5]:
import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask, jsonify, make_response, request
import threading

app = Flask(__name__)
padding_size = 1000
model = tf.keras.models.load_model('/content/drive/My Drive/sentiment_model/sentiment_analysis.hdf5')
text_encoder = tfds.features.text.TokenTextEncoder.load_from_file("/content/drive/My Drive/sentiment_model/sa_encoder.vocab")

print('Model and Vocabalory loaded.......')

Model and Vocabalory loaded.......


In [0]:
@app.route("/")
def hello():
    return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec

def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())

In [0]:
@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})

In [8]:
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':6000}).start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:6000/ (Press CTRL+C to quit)


In [9]:
import requests
req = requests.get("http://172.28.0.2:6000/")
print(req.status_code)
print(req.text)

172.28.0.2 - - [10/Jun/2020 17:15:03] "GET / HTTP/1.1" 200 -


200
I am alive!


In [10]:
import requests, json
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"Still working my way through it but definitely changes your view on investment. Wish it was available on Audible"}
req = requests.post("http://172.28.0.2:6000/seclassifier",  data=json.dumps(data), headers=headers)

Still working my way through it but definitely changes your view on investment. Wish it was available on Audible


172.28.0.2 - - [10/Jun/2020 17:15:17] "POST /seclassifier HTTP/1.1" 200 -


In [11]:
print(req.status_code)
print(req.text)

200
{"predictions ":[[0.8370056748390198]],"sentiment ":"positive"}



In [12]:
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"I thought this will be good one. But I was wrong"}
req = requests.post("http://172.28.0.2:6000/seclassifier",  data=json.dumps(data), headers=headers)

I thought this will be good one. But I was wrong


172.28.0.2 - - [10/Jun/2020 17:15:45] "POST /seclassifier HTTP/1.1" 200 -


In [13]:
print(req.status_code)
print(req.text)

200
{"predictions ":[[-0.8676939606666565]],"sentiment ":"Negative"}



# Running Flask as a External Server using ngrok

In [14]:
import flask
flask.__version__

'1.1.2'

In [0]:
!pip -q install flask-ngrok

In [18]:


import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask, jsonify, make_response, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

padding_size = 1000
model = tf.keras.models.load_model('/content/drive/My Drive/sentiment_model/sentiment_analysis.hdf5')
text_encoder = tfds.features.text.TokenTextEncoder.load_from_file("/content/drive/My Drive/sentiment_model/sa_encoder.vocab")

print('Model and Vocabalory loaded.......')

@app.route("/")
def hello():
    return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec


def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})


Model and Vocabalory loaded.......


In [19]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://214dd9884363.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [10/Jun/2020 17:30:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jun/2020 17:30:27] "GET /favicon.ico HTTP/1.1" 404 -
